In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
import pandas as pd
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

In [90]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


Thhe point of the intervention only analysis is that we have already done the intervention analysis in an R linear model. We should be able to reproduce that result here. So if we don't, we can work on debugging what might be wrong with the pipeline.

# Get the data

## Set up data


In [91]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
#icvm.mode = 'full_pipeline_test'
icvm.mode = 'fast_analysis'
#icvm.mode = 'full_analysis'

#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV022', 'DEV002', 'DEV280', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(243, 33)
(243, 33)


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [93]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV004,umpqua
1,DEV005,umpqua
2,DEV008,umpqua
3,DEV009,umpqua
4,DEV010,mckenzie
...,...,...
238,DEV308,willamette
239,DEV309,umpqua
240,DEV310,mckenzie
241,DEV311,willamette


In [94]:
dev_cv_analysis.outcome_measures.columns

Index(['SID', 'bf', 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage',
       'ANTINUTRIENT_DENSITY_2wkAverage', 'total_calorie'],
      dtype='object')

In [95]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_only': condition_cols
}


outcome_vars_to_try = [ 'bf','cancer_promoting_FFQ',
       'NUTRIENT_DENSITY_2wkAverage']

In [96]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

EDM, BIS_11, PCS, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, NCS_total, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_FCI, age365, education_own, household_income_per_person, SST_PostErrorSlowW1_mean, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, ROC_Crave_Minus_Neutral, WTP_unhealthy_minus_healthy, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, sst_CorrectGo_striatum_joint_mask, sst_FailedStop_motor_control_striatum_joint_mask, sst_CorrectGoFollowingFailedStop_striatum_joint_mask, Planning_aggregate, Restraint_aggregate, IMI_effort_importance_aggregate, wtp_roc_koban_kober_craving_combined, birthsex_factor_Male, umpqua, mckenzie, EDM*umpqua, BIS_11*umpqua, PCS*umpqua, ACES_sum*umpqua, BFI_agreeableness*umpqua, BFI_conscientiousness*umpqua, BFI_extraversion*umpqua, BFI_neurotic

In [97]:
print(", ".join(dev_cv_analysis.outcome_measures.columns))

SID, bf, NUTRIENT_RICH_FOODS_INDEX_2wkAverage, ANTINUTRIENT_DENSITY_2wkAverage, total_calorie


## Total calories

In [98]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'total_calorie', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV022', 'DEV002', 'DEV280', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(243, 33)
(243, 33)
 attempting to predict total_calorie with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipelin

doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
0,umpqua,None,-0.000404,0.000404
1,mckenzie,None,-0.000131,0.000131


{'condition_only': -0.027494300215936684}


## ANTINUTRIENT_DENSITY_2wkAverage

In [99]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'ANTINUTRIENT_DENSITY_2wkAverage', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV022', 'DEV002', 'DEV280', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(243, 33)
(243, 33)
 attempting to predict ANTINUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totall

KeyboardInterrupt: 

## NUTRIENT_DENSITY_2wkAverage

In [ ]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV022', 'DEV002', 'DEV280', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(243, 33)
(243, 33)
 attempting to predict NUTRIENT_RICH_FOODS_INDEX_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, t

doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
0,umpqua,None,0.048786,0.048786
1,mckenzie,None,0.016656,0.016656


{'condition_only': 0.01388523966495181}


## Body Fat Percentage

In [ ]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_DENSITY_2wkAverage')

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV032', 'DEV022', 'DEV002', 'DEV280', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(243, 33)
(243, 33)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are umpqua mckenzie
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 

doing permutation test on importance; this may take time.
Number of selected features: 2


,predictor,coef,feature_importance,fa_abs
0,umpqua,2.316367,0.041563,0.041563
1,mckenzie,-0.139958,0.001002,0.001002


{'condition_only': -0.002032553737788656}


Before we dive in to debug the analysis, let's try a statsmodel approach. We can use statsmodel to access the same data the ML is doing to see how it goes in identifying coefficients for umpqua, mckenzie, and willamette.

In [ ]:
dev_cv_analysis.outcome_measures

,SID,bf,NUTRIENT_DENSITY_2wkAverage,ANTINUTRIENT_DENSITY_2wkAverage,total_calorie
0,DEV004,-0.6,NaN,NaN,NaN
1,DEV005,0.4,NaN,NaN,NaN
2,DEV008,-1.1,NaN,NaN,NaN
3,DEV009,-1.6,NaN,NaN,NaN
4,DEV010,0.2,39.286168,NaN,5351.104167
...,...,...,...,...,...
238,DEV308,-0.4,18.115014,12.426329,4098.762500
239,DEV309,-0.5,15.260742,-2.335843,-4498.379167
240,DEV310,0.1,-1.960706,-5.855379,-290.115000
241,DEV311,0.5,42.269001,-2.790664,-4973.492499


# Using statsmodel

In [ ]:
#import the statsmodel package
import statsmodels.api as sm

#do a statsmodel linear regression
X_onehots_raw = dev_cv_analysis.group_assignment_onehots
y_raw = dev_cv_analysis.outcome_measures['NUTRIENT_RICH_FOODS_INDEX_2wkAverage']
na_mask = y_raw.isna() | X_onehots_raw.isna().any(axis=1)
X_onehots = X_onehots_raw[~na_mask]
y = y_raw[~na_mask]

X_onehots.to_csv(dev_cv_analysis.dropbox_data_dir + "/intervention_only_X_onehots.csv")
y.to_csv(dev_cv_analysis.dropbox_data_dir + "/intervention_only_y.csv")

nutrient_density_ols_model = sm.OLS(y, X_onehots).fit()




In [ ]:
nutrient_density_ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                          OLS Regression Results                                         
=========================================================================================================
Dep. Variable:     NUTRIENT_RICH_FOODS_INDEX_2wkAverage   R-squared (uncentered):                   0.111
Model:                                              OLS   Adj. R-squared (uncentered):              0.103
Method:                                   Least Squares   F-statistic:                              13.57
Date:                                  Fri, 14 Jul 2023   Prob (F-statistic):                    2.80e-06
Time:                                          17:24:08   Log-Likelihood:                         -942.77
No. Observations:                                   220   AIC:                                      1890.
Df Residuals:                                       218   BIC:                                      1896.
Df Model:                                             2                                                  
Covariance Type:                              nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
umpqua        11.0091      2.141      5.143      0.000       6.790      15.228
mckenzie       1.6449      1.986      0.828      0.408      -2.270       5.559
==============================================================================
Omnibus:                        3.470   Durbin-Watson:                   1.888
Prob(Omnibus):                  0.176   Jarque-Bera (JB):                3.720
Skew:                          -0.116   Prob(JB):                        0.156
Kurtosis:                       3.593   Cond. No.                         1.08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Compared to the model in R, the t-score and p-value are almost identical, but the R-squared and adjusted r-squared look pretty different.

In [ ]:
#add a constant to the model
X_onehots_w_const = sm.add_constant(X_onehots)
nutrient_density_ols_model = sm.OLS(y, X_onehots_w_const).fit()
nutrient_density_ols_model.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     NUTRIENT_RICH_FOODS_INDEX_2wkAverage   R-squared:                       0.051
Model:                                              OLS   Adj. R-squared:                  0.042
Method:                                   Least Squares   F-statistic:                     5.819
Date:                                  Fri, 14 Jul 2023   Prob (F-statistic):            0.00345
Time:                                          17:24:11   Log-Likelihood:                -941.62
No. Observations:                                   220   AIC:                             1889.
Df Residuals:                                       217   BIC:                             1899.
Df Model:                                             2                                         
Covariance Type:                              nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.1090      2.060      1.509      0.133      -0.951       7.169
umpqua         7.9000      2.967      2.663      0.008       2.053      13.747
mckenzie      -1.4641      2.858     -0.512      0.609      -7.096       4.168
==============================================================================
Omnibus:                        3.402   Durbin-Watson:                   1.881
Prob(Omnibus):                  0.183   Jarque-Bera (JB):                3.768
Skew:                          -0.085   Prob(JB):                        0.152
Kurtosis:                       3.618   Cond. No.                         3.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

hmmm--with a constant, the effect just about disappears. That's really odd, because we'd expect the constant to behave like the constant in R's lm function.

Do we have the same dataset? small differences in the dataset can make a big difference.

In [ ]:
outcomes_na_masked = dev_cv_analysis.outcome_measures[~na_mask]

In [ ]:
outcomes_na_masked

,SID,bf,NUTRIENT_DENSITY_2wkAverage,ANTINUTRIENT_DENSITY_2wkAverage,total_calorie
4,DEV010,0.2,39.286168,NaN,5351.104167
5,DEV011,-0.8,19.513277,-17.041968,-4748.649166
6,DEV012,0.5,7.317783,8.130856,-1419.556666
8,DEV014,-2.3,19.641489,0.990242,-2743.742500
9,DEV015,0.5,7.616559,0.354070,1683.897500
...,...,...,...,...,...
238,DEV308,-0.4,18.115014,12.426329,4098.762500
239,DEV309,-0.5,15.260742,-2.335843,-4498.379167
240,DEV310,0.1,-1.960706,-5.855379,-290.115000
241,DEV311,0.5,42.269001,-2.790664,-4973.492499


In [ ]:
", ".join(outcomes_na_masked.SID)

'DEV010, DEV011, DEV012, DEV014, DEV015, DEV016, DEV017, DEV018, DEV020, DEV021, DEV023, DEV024, DEV025, DEV026, DEV027, DEV028, DEV029, DEV030, DEV031, DEV033, DEV035, DEV036, DEV038, DEV039, DEV040, DEV041, DEV042, DEV043, DEV044, DEV046, DEV047, DEV048, DEV049, DEV050, DEV051, DEV052, DEV053, DEV054, DEV055, DEV056, DEV057, DEV058, DEV059, DEV060, DEV061, DEV062, DEV063, DEV064, DEV065, DEV066, DEV067, DEV068, DEV069, DEV070, DEV071, DEV072, DEV073, DEV074, DEV075, DEV076, DEV077, DEV078, DEV079, DEV083, DEV084, DEV085, DEV086, DEV087, DEV088, DEV089, DEV090, DEV091, DEV093, DEV094, DEV096, DEV097, DEV098, DEV099, DEV100, DEV101, DEV102, DEV103, DEV104, DEV107, DEV108, DEV109, DEV110, DEV112, DEV113, DEV115, DEV116, DEV117, DEV118, DEV119, DEV120, DEV121, DEV122, DEV125, DEV126, DEV127, DEV128, DEV129, DEV130, DEV132, DEV133, DEV134, DEV135, DEV137, DEV138, DEV139, DEV140, DEV141, DEV143, DEV144, DEV145, DEV147, DEV149, DEV151, DEV153, DEV155, DEV156, DEV157, DEV158, DEV159, DEV161,

In [ ]:
outcomes_na_masked.loc[:,['SID','NUTRIENT_DENSITY_2wkAverage']]

,SID,NUTRIENT_DENSITY_2wkAverage
4,DEV010,39.286168
5,DEV011,19.513277
6,DEV012,7.317783
8,DEV014,19.641489
9,DEV015,7.616559
...,...,...
238,DEV308,18.115014
239,DEV309,15.260742
240,DEV310,-1.960706
241,DEV311,42.269001


In [ ]:

data_by_wave_ppt = pd.read_csv(dev_cv_analysis.data_by_wave_ppt_path)

In [ ]:
#apply standardscaler to the y variables
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.values.reshape(-1,1))



In [ ]:
#add a constant to the model
nutrient_density_ols_model = sm.OLS(y_scaled, X_onehots).fit()
nutrient_density_ols_model.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.047
Model:                            OLS   Adj. R-squared (uncentered):              0.038
Method:                 Least Squares   F-statistic:                              5.390
Date:                Fri, 14 Jul 2023   Prob (F-statistic):                     0.00519
Time:                        17:25:00   Log-Likelihood:                         -306.86
No. Observations:                 220   AIC:                                      617.7
Df Residuals:                     218   BIC:                                      624.5
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
umpqua         0.3335      0.119      2.804      0.005       0.099       0.568
mckenzie      -0.1884      0.110     -1.707      0.089      -0.406       0.029
==============================================================================
Omnibus:                        3.304   Durbin-Watson:                   1.873
Prob(Omnibus):                  0.192   Jarque-Bera (JB):                3.690
Skew:                          -0.065   Prob(JB):                        0.158
Kurtosis:                       3.621   Cond. No.                         1.08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This is all interesting, but Ridge and Lasso both add an intercept by default, so we don't need to worry about it.